# Cleaning data
Oh the humanity! The way some folks treat their data is shocking and we usually need to clean it up before we can even start to think about reasonable visualisations. Let's take a look at some of our options. Note that since people do many cruel and usual things to do their data this is by no means an exhaustive section!

In [1]:
install.packages(c("tidyverse","skimr"))
library(tidyverse)
heroes = read_csv("data/heroes_information.csv")
powers = read_csv("data/super_hero_powers.csv")

Installing packages into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_integer(),
  name = col_character(),
  Gender = col_character(),
  `Eye color` = col_character(),
  Race = col_character(),
  `Hair color` = col_character(),
  Height = col_double(),
  Publisher = col_character(),
  `Skin color` = col_character(),
  Alignment = col_character(),
  Weight = col_double()
)
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


## Working with missings
Missing values can be a real pain, whether there's lots of them, just a few frustrating values, or codes instead of real missings.

Let's looks at soome tools and code techniques to help us handle these cases.

## Exploring missings
Usually we need to first of all explore missing values first. The `naniar` package is excellent for this. Make sure to [read the docs](https://cran.r-project.org/web/packages/naniar/vignettes/getting-started-w-naniar.html) to dive deeper into this package at a later date.

In [2]:
install.packages("naniar")
library(naniar)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [3]:
gg_miss_var(heroes) + ggthemes::theme_few()  #Theme is a sneak peek of some stuff we'll be using later

### Replacing missings
Particularly with text columns, we often want to turn any missing values into an explicit missing category. The `forcats` package (part of the `tidyverse` so already loaded!) has a function that will change `NA` vallues to a text value for us. We can also use some of the fancy data manipulation syntax from the `tidyverse` to do this for **every** text column in our dataset. IN A SINGLE LINE OF CODE!!

In [4]:
# mutate_if takes a dataset, applies a condition to each column, and for each column that returned TRUE applies a function to it
heroes = mutate_if(heroes, is.character, fct_explicit_na)
gg_miss_var(heroes) + ggthemes::theme_few() 

## Removing codes
The other option we want to frequently do is remove the codes like `-99` that stand for `NA` in our data. We can use our `mutate_if()` super function to remove these values for all our numeric columns. We need to do a little bit of work though to make a useful helper function to keep our code simple.

We can make our own reusable bits of logic from simple column-level things to complex pieces that grab data from the internet and process it. Learn more about [making functions](http://adv-r.had.co.nz/Functions.html) at a later date.

In [7]:
replaceLT0 = function(x) ifelse(x < 0, NA, x) # This is like an Excel if statement that will do the condition for every row
heroes = mutate_if(heroes, is.numeric, replaceLT0)
gg_miss_var(heroes) + ggthemes::theme_few() 

>Care to try your hand doing some text clean up? There's hyphens being used to denote NA in the `heroes` data. Turn these into NAs

## Tidying values
We'll often need to do text, date, and numeric manipulation to clean up values. Key packages that we have because of the `tidyverse` are:
- `stringr` for working with text
- `forcats` for working with text that has fairly low amounts of unique values
- `lubridate` for working with dates

This is an area where we unfortunately spend a lot of time!There's tons of resources about it on the interwebs thankfully.

One particular issue we *definitely* need to clean up right now is the fact that there's a bunch of super power data that has power columns with the values `True` and `False` as  character strings for many, many columns. 

An alternative to our trusty `mutate_if()` is `mutate_at()` which will allow us to say what range of columns we want to process. Oh, and I forgot to mention before because we're killing it with amazing coding multipliers but there's also a `mutate()` function you can use for changing or adding columns when you need to think about a few named columns.

In [8]:
boolConversion = function(x) x == "True"
# mutate_at needs to know which variables to include or exclude. HEre I'm using the super nifty exclude option
powers = mutate_at(powers, vars(-hero_names), boolConversion)
str(powers)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	667 obs. of  168 variables:
 $ hero_names                  : chr  "3-D Man" "A-Bomb" "Abe Sapien" "Abin Sur" ...
 $ Agility                     : logi  TRUE FALSE TRUE FALSE FALSE FALSE ...
 $ Accelerated Healing         : logi  FALSE TRUE TRUE FALSE TRUE FALSE ...
 $ Lantern Power Ring          : logi  FALSE FALSE FALSE TRUE FALSE FALSE ...
 $ Dimensional Awareness       : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ Cold Resistance             : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ Durability                  : logi  FALSE TRUE TRUE FALSE FALSE FALSE ...
 $ Stealth                     : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ Energy Absorption           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ Flight                      : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ Danger Sense                : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ Underwater breathing        : logi  FALSE FALSE TRUE FALSE FALS

## Reshaping data 
Often we get data in a summary form that makes it hard to be able to do visualisations and analysis. 

It's usually better if we can get our data into a *long* format where instead of lots of columns, we have lots of rows instead. This is what I think of as *unpivoting* my data because *pivoting* it means ending up with many columns.

In R, the function for unpivoting data is `gather()`, like gathering all your data up, and the function for pivoting your data is `spread()`.

`gather()` will need to know what the name should be for column containing our old headers, what the column name should be for the one holding our old cell values, and what columns we do/don't want to unpivot.

In [9]:
powers_long = gather(powers, power, present, -hero_names)
str(powers_long)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	111389 obs. of  3 variables:
 $ hero_names: chr  "3-D Man" "A-Bomb" "Abe Sapien" "Abin Sur" ...
 $ power     : chr  "Agility" "Agility" "Agility" "Agility" ...
 $ present   : logi  TRUE FALSE TRUE FALSE FALSE FALSE ...


> Have a think about what might be needed to re-pivot the data (feel free to peek at the docs too!) and give it a go

## Storing intermediate results and outputs
We saw in a previous section that we could write to a directory, but we did a **bad thing** as we wrote raw data to an outputs directory. It's good practice to keep your data in three directories resembling level of processing:
- `raw` (or just `data` depending on you want to name things) for the data as you received it
- `clean` (or `intermediate` etc) for data that you've done cleaning steps on and is now fit for use
- `outputs` (or `ssummaries`) for processed and refined things like summaries, specific extracts and so on

In [11]:
dir.create("clean", showWarnings = FALSE)
write_csv(heroes, "clean/heroes.csv")
write_csv(powers, "clean/powers.csv")
write_csv(powers_long, "clean/powers_long.csv")